### Sources

- https://data.sfgov.org/Geographic-Locations-and-Boundaries/Bay-Area-Counties/s9wg-vcph
- https://gis.stackexchange.com/questions/314949/creating-square-buffers-around-points-using-shapely
- https://geopandas.org/gallery/create_geopandas_from_pandas.html
- http://geologyandpython.com/ml-interpolation-method.html
- https://www.arb.ca.gov/aqmis2/display.php?param=PM25HR&units=001&year=2020&report=PICKDATA&mon=3&day=27&o3area=&o3pa8=&county_name=--COUNTY--&latitude=--PART+OF+STATE--&basin=SFB-San+Francisco+Bay&order=basin,county_name,s.name&ptype=aqd


In [1]:
import geopandas as gpd
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import json
from shapely.geometry import Polygon, Point
from scipy.interpolate import griddata
import numpy as np
import requests
import io
from datetime import date
alt.renderers.enable('default')

RendererRegistry.enable('default')

In [2]:
# Load site information
sites = pd.read_csv('../data/raw/PM25HR_SITELIST_2020-12-31.csv')
sites.astype({'site': 'str'})

# Add locations of air quality stations in lat/long
sites_wr = sites.groupby(['site', 'name']).aggregate({'latitude':'mean',
                                            'longitude':'mean'}).reset_index()
# Cast 'site' column as a string to match data df
sites_wr['site'] = sites_wr['site'].astype('int').astype('str')

# Export for plotting
sites_wr.to_csv('../data/wrangled/sites_data.csv')

## Run per pollutant to create df for plotting

In [3]:
# Choose pollutant
param = 'PM25HR' # OZONE, BC, NOX, PM25HR

# Current date
today = date.today()

# Set parameters
first_date = '2020-2-15' # yyyy-m-d 
units = {'OZONE': '007', 'BC': '001', 'NOX': '007', 'PM25HR': '001'}
year = '2020'
mon = str(today.month)
day = str(today.day)
basin = 'SFB-San+Francisco+Bay'
rows = {'OZONE': '20', 'BC': '6', 'NOX': '18', 'PM25HR': '17'}
fname = param+'_'+year+'-'+mon+'-'+day

url = 'https://www.arb.ca.gov/aqmis2/display.php?sitelist=All&filefmt=csv&\
fname='+fname+'&datafmt=web&download=y&first_date='+first_date+'&\
param='+param+'&units='+units[param]+'&year='+year+'&report=PICKDATA&mon='+mon+'&day='+day+'&o3area=&o3pa8=&\
county_name=--COUNTY--&latitude=--PART+OF+STATE--&basin='+basin+'&\
order=basin%2Ccounty_name%2Cs.name&ptype=aqd&o3switch=new&hours=all&statistic=&\
qselect=Screened&start_mon=2&start_day=1&submit=All+Sites&rows='+rows[param]

r = requests.post(url)
if r.ok:
    data = r.content.decode('utf8')
    aq_data = pd.read_csv(io.StringIO(data))    
#pm25_data = pd.read_csv('../data/raw/PM25HR_PICKDATA_2020-3-27.csv')

# Remove observations marked "invalid"
aq_data = aq_data.dropna()

# Map latitude, longitude values to new columns in data df based on site id
lat_dict = dict(zip(sites_wr['site'], sites_wr['latitude']))
long_dict = dict(zip(sites_wr['site'], sites_wr['longitude']))
aq_data['lat'] = aq_data['site'].map(lat_dict)
aq_data['long'] = aq_data['site'].map(long_dict)


In [4]:
# Cast sites as geodataframe
gdf_sites = gpd.GeoDataFrame(
    sites_wr, geometry=gpd.points_from_xy(sites_wr.longitude, sites_wr.latitude))

# Reshape df, column per date
aq_da = aq_data[['site', 'date', 'value']].groupby(['site','date']).mean().reset_index()
aq_da = aq_da.pivot(index = 'site', values = 'value', columns = 'date').reset_index()
# Merge spatial data with aq data
aq_gdf = pd.merge(gdf_sites, aq_da, how = 'inner', on = 'site')

# Create grid to interpolate across

# Define size of pixels in grid (units of lat/long degrees)
pixel = .08
# Determine extent of observations and create pixel_size-spaced arrays
# in the N and S direction
x_range = np.arange(aq_gdf.longitude.min() - aq_gdf.longitude.min() % pixel,
                    aq_gdf.longitude.max(), pixel)
y_range = np.arange(aq_gdf.latitude.min() - aq_gdf.latitude.min() % pixel,
                    aq_gdf.latitude.max(), pixel)[::-1]
shape = (len(y_range), len(x_range))
xmin, xmax, ymin, ymax = x_range.min(), x_range.max(), y_range.min(), y_range.max()
extent = (xmin, xmax, ymin, ymax)
# Create grid
x_mesh, y_mesh = np.meshgrid(x_range, y_range)

# Create dataframe to store interpolated points in
interp_df = pd.DataFrame({'lat':y_mesh.flatten(), 'long': x_mesh.flatten()})

date_cols = aq_gdf.columns[5:]
for date in date_cols[15:]:
    pm_interp = griddata((aq_gdf['longitude'], aq_gdf['latitude']), aq_gdf[date], (x_mesh, y_mesh), method = 'linear')
    interp_df[date] = pm_interp.flatten()

# Cast to geodataframe
interp_gdf = gpd.GeoDataFrame(
    interp_df, geometry=gpd.points_from_xy(interp_df.long, interp_df.lat))

# Buffer each grid point to a larger pixel
interp_gdf = gpd.GeoDataFrame(
    interp_df, geometry=gpd.points_from_xy(interp_df.long, interp_df.lat))

interp_buffer = interp_gdf.copy()
buffer = interp_buffer.buffer(0.04)
envelope = buffer.envelope
interp_buffer['geometry'] = envelope

# Write csv with interpolated values
# interp_buffer = interp_buffer.drop(columns = ['x_idx','y_idx'])
interp_buffer = interp_buffer.melt(id_vars = ['geometry','lat','long'], 
                                   var_name = 'date', 
                                   value_name = param)
interp_buffer['date_int'] = interp_buffer['date'].map(dict(zip(date_cols, range(len(date_cols)))))
interp_buffer

,geometry,lat,long,date,PM25HR,date_int
0,"POLYGON ((-122.92000 38.36000, -122.84000 38.3...",38.40,-122.88,2020-03-01,NaN,15
1,"POLYGON ((-122.84000 38.36000, -122.76000 38.3...",38.40,-122.80,2020-03-01,NaN,15
2,"POLYGON ((-122.76000 38.36000, -122.68000 38.3...",38.40,-122.72,2020-03-01,NaN,15
3,"POLYGON ((-122.68000 38.36000, -122.60000 38.3...",38.40,-122.64,2020-03-01,NaN,15
4,"POLYGON ((-122.60000 38.36000, -122.52000 38.3...",38.40,-122.56,2020-03-01,NaN,15
...,...,...,...,...,...,...
9039,"POLYGON ((-121.96000 36.92000, -121.88000 36.9...",36.96,-121.92,2020-03-28,NaN,42
9040,"POLYGON ((-121.88000 36.92000, -121.80000 36.9...",36.96,-121.84,2020-03-28,NaN,42
9041,"POLYGON ((-121.80000 36.92000, -121.72000 36.9...",36.96,-121.76,2020-03-28,NaN,42
9042,"POLYGON ((-121.72000 36.92000, -121.64000 36.9...",36.96,-121.68,2020-03-28,NaN,42


In [5]:
line_df = interp_buffer.copy().dropna()
line_df['Lat_Long'] = interp_buffer['lat'].astype(str)+interp_buffer['long'].astype(str)
region_avg = interp_buffer.iloc[:,3:-1].dropna().groupby('date').mean().reset_index()

In [6]:
alt.Chart(region_avg).mark_line().encode(x='date:T',y=param)

alt.Chart(...)